In [1]:
import os
import pandas as pd 
import numpy as np 
import Levenshtein
import utils
import cleaning

In [2]:
df1 = pd.read_csv(os.path.join('zippedData', 'tn.movie_budgets.csv.gz'))
df1['release_date'] = pd.to_datetime(df1['release_date'])
# df1['release_year'] = df1['release_date'].dt.year
df1

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,2018-12-31,Red 11,"$7,000",$0,$0
5778,79,1999-04-02,Following,"$6,000","$48,482","$240,495"
5779,80,2005-07-13,Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,2015-09-29,A Plague So Pleasant,"$1,400",$0,$0


In [3]:
df2 = pd.read_csv(os.path.join('zippedData', 'bom.movie_gross.csv.gz'))
df2

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [4]:
df1_titles = cleaning.process_strings(df1.movie).values
df2_titles = cleaning.process_strings(df2.title).values

In [5]:
# titles = np.tile(df2_titles, (df1_titles.size, 1)).T
titles = pd.DataFrame(index=df2_titles, columns=df1_titles)
titles.head()

,avatar,pirates of the caribbean on stranger tides,dark phoenix,avengers age of ultron,star wars ep viii the last jedi,star wars ep vii the force awakens,avengers infinity war,pirates of the caribbean at worldas end,justice league,spectre,...,newlyweds,el mariachi,primer,cavite,the mongol king,red 11,following,return to the land of wonders,a plague so pleasant,my date with drew
toy story 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
alice in wonderland 2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
harry potter and the deathly hallows part 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
inception,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shrek forever after,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def similarity(data: pd.Series, base_str=None) -> pd.Series:
    strings = data.index.to_series()
    if base_str is None:
        base_str = data.name
    return strings.map(lambda x: Levenshtein.ratio(base_str, x))

In [7]:
sim_df = titles.apply(similarity)

In [10]:
sim_df

,avatar,pirates of the caribbean on stranger tides,dark phoenix,avengers age of ultron,star wars ep viii the last jedi,star wars ep vii the force awakens,avengers infinity war,pirates of the caribbean at worldas end,justice league,spectre,...,newlyweds,el mariachi,primer,cavite,the mongol king,red 11,following,return to the land of wonders,a plague so pleasant,my date with drew
toy story 3,0.235294,0.264151,0.173913,0.242424,0.238095,0.311111,0.250000,0.280000,0.240000,0.333333,...,0.200000,0.181818,0.117647,0.117647,0.307692,0.235294,0.200000,0.300000,0.258065,0.285714
alice in wonderland 2010,0.133333,0.363636,0.277778,0.304348,0.327273,0.310345,0.355556,0.412698,0.315789,0.129032,...,0.242424,0.228571,0.200000,0.200000,0.307692,0.266667,0.181818,0.377358,0.363636,0.341463
harry potter and the deathly hallows part 1,0.204082,0.400000,0.254545,0.307692,0.432432,0.415584,0.343750,0.390244,0.175439,0.160000,...,0.230769,0.185185,0.163265,0.122449,0.206897,0.204082,0.192308,0.416667,0.349206,0.366667
inception,0.133333,0.235294,0.285714,0.322581,0.200000,0.232558,0.266667,0.250000,0.260870,0.250000,...,0.222222,0.300000,0.266667,0.266667,0.250000,0.133333,0.222222,0.210526,0.206897,0.153846
shrek forever after,0.320000,0.393443,0.322581,0.390244,0.440000,0.415094,0.350000,0.344828,0.363636,0.307692,...,0.142857,0.266667,0.240000,0.240000,0.294118,0.240000,0.142857,0.416667,0.307692,0.277778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
the quake,0.266667,0.235294,0.285714,0.258065,0.300000,0.325581,0.200000,0.250000,0.434783,0.250000,...,0.222222,0.300000,0.133333,0.266667,0.416667,0.266667,0.000000,0.315789,0.206897,0.307692
edward ii 2018 rerelease,0.133333,0.333333,0.277778,0.260870,0.436364,0.413793,0.311111,0.349206,0.315789,0.193548,...,0.303030,0.285714,0.266667,0.200000,0.153846,0.266667,0.121212,0.301887,0.363636,0.341463
el pacto,0.285714,0.160000,0.300000,0.333333,0.205128,0.190476,0.206897,0.212766,0.272727,0.400000,...,0.235294,0.526316,0.142857,0.285714,0.260870,0.285714,0.235294,0.216216,0.357143,0.240000
the swan,0.285714,0.280000,0.300000,0.266667,0.256410,0.333333,0.275862,0.297872,0.363636,0.266667,...,0.235294,0.315789,0.142857,0.285714,0.434783,0.285714,0.235294,0.324324,0.357143,0.320000


In [8]:
match_df = sim_df.agg(['idxmax', 'max'], axis=1)
match_df.shape

(3387, 2)

In [13]:
match_df[match_df['max'] >= 0.95].sort_values('max')

,idxmax,max
i spit on your grave 2,i spit on your grave,0.952381
spiderman into the spiderverse,spiderman into the spiderverse 3d,0.952381
the pirates,the pirate,0.952381
the players,the player,0.952381
piranha 3dd,piranha 3d,0.952381
...,...,...
moonrise kingdom,moonrise kingdom,1
the watch,the watch,1
anna karenina,anna karenina,1
to rome with love,to rome with love,1


In [15]:
match_df[match_df['max'] == 1].sort_values('max')

,idxmax,max
toy story 3,toy story 3,1
woman in gold,woman in gold,1
brooklyn,brooklyn,1
the longest ride,the longest ride,1
unfriended,unfriended,1
...,...,...
what to expect when youre expecting,what to expect when youre expecting,1
the possession,the possession,1
this is 40,this is 40,1
the three stooges,the three stooges,1
